In [16]:
!pip install streamlit pyngrok

In [17]:
# Replace YOUR_TOKEN_HERE with your actual Ngrok token
!ngrok config add-authtoken ENTER YOUR KEY


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [18]:
import os
os.makedirs("model", exist_ok=True)
os.makedirs("bgs", exist_ok=True)

In [19]:
with open("model/labels.txt", "w") as f:
    f.write("0 Pneumonia\n")
    f.write("1 Normal\n")

In [20]:
from google.colab import files
uploaded = files.upload()  # Upload bg5.png manually
!mv bg5.png bgs/

Saving bg5.png to bg5.png


In [21]:
from google.colab import files
uploaded = files.upload()  # Upload pneumonia_classifier.h5
!mv pneumonia_classifier.h5 model/

Saving pneumonia_classifier.h5 to pneumonia_classifier.h5


In [22]:
%%writefile util.py
import base64
import streamlit as st
from PIL import ImageOps, Image
import numpy as np

def set_background(image_file):
    with open(image_file, "rb") as f:
        img_data = f.read()
    b64_encoded = base64.b64encode(img_data).decode()
    style = f"""
        <style>
        .stApp {{
            background-image: url(data:image/png;base64,{b64_encoded});
            background-size: cover;
        }}
        </style>
    """
    st.markdown(style, unsafe_allow_html=True)

def classify(image, model, class_names):
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array
    prediction = model.predict(data)
    index = 0 if prediction[0][0] > 0.95 else 1
    class_name = class_names[index]
    confidence_score = prediction[0][index]
    return class_name, confidence_score


Overwriting util.py


In [23]:
%%writefile main.py
import streamlit as st
from keras.models import load_model
from PIL import Image
from util import classify, set_background

# Set custom background
set_background('./bgs/bg5.png')

# Set title and header
st.title('Pneumonia classification')
st.header('Please upload a chest X-ray image')

# Upload image
file = st.file_uploader('', type=['jpeg', 'jpg', 'png'])

# Load model
model = load_model('./model/pneumonia_classifier.h5')

# Load class names
with open('./model/labels.txt', 'r') as f:
    class_names = [a[:-1].split(' ')[1] for a in f.readlines()]
    f.close()

# Process and classify uploaded image
if file is not None:
    image = Image.open(file).convert('RGB')
    st.image(image, use_column_width=True)
    class_name, conf_score = classify(image, model, class_names)
    st.write("## {}".format(class_name))
    st.write("### score: {}%".format(int(conf_score * 1000) / 10))


Overwriting main.py


In [24]:
# Start Streamlit in the background
!nohup streamlit run main.py &> /content/logs.txt &


In [25]:
# Open Ngrok tunnel (for public link)
from pyngrok import ngrok
import time

time.sleep(5)
public_url = ngrok.connect(addr="8501", proto="http")
print("🌐 Your app is live at:", public_url)


🌐 Your app is live at: NgrokTunnel: "https://21bd-34-106-152-252.ngrok-free.app" -> "http://localhost:8501"
